<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 5. Reactive Pipelines</h2></div>

The plots built up over the first few tutorials were already highly interactive in the web browser, with interactive tools and events in each plot, and sometimes Bokeh widgets created by HoloViews automatically to select a specific categorical value from a `groupby`. However, when you are exploring a dataset, the automatic interactivity may not be enough -- you may want to add interactive control to some other aspect of how data is selected or generated or how it is visualized that _you_ decide. Luckily, HoloViz makes it almost trivially easy to add any interactive behavior that you want, making it easy for you or your users to explore any parameter or setting in your code. 

Though we haven't yet needed to discuss it, the Bokeh widgets generated by hvPlot were configured using the [Panel](https://panel.holoviz.org) library from HoloViz, which focuses on widgets, layouts, and constructing standalone HoloViz applications. In this notebook, we'll use Panel's support for the "reactive expressions" from the underlying [Param](https://param.holoviz.org/en/docs/latest/user_guide/Reactive_Expressions.html) library to add just about any interactivity you can imagine. E.g. if you want to control  filtering or selecting or aggregation in your Pandas or GeoPandas or Dask DataFrame, or Xarray Dataset, you simply wrap that object in `pn.rx()`. `pn.rx()` returns an reactive object (e.g. a reactive Pandas DataFrame) that can be used almost exactly the same as the original object (e.g. calling regular Pandas methods) but whose *output* (e.g. a DataFrame view) will be re-computed everytime one of the reactive pipeline *inputs* change. The *inputs* are typically widgets (e.g. a drop-down list) that replace values you would usually hard-code and manually update to observe how they affect the output. When such an reactive object is displayed in a notebook, it includes the widgets that you have used together with the regular output, and the output is automatically updated appropriately.

## Panel widgets

Before using `rx()` we will need some widgets. You can use [ipywidgets](https://ipywidgets.readthedocs.io) if you want, but to keep things consistent with hvPlot's widgets, here we will be using Bokeh widgets from [Panel](https://panel.holoviz.org/). Let's first get ahold of a Panel widget to see how they work. Here, let's create a Panel floating-point number slider to specify an earthquake magnitude between zero and nine:

In [ ]:
import pathlib

import holoviews as hv
import hvplot.pandas # noqa
import numpy as np
import pandas as pd
import panel as pn

pn.extension(throttled=True)

In [ ]:
mag_slider = pn.widgets.FloatSlider(name='Minimum Magnitude', start=0, end=9, value=6)
mag_slider

The widget is a JavaScript object, but there are bidirectional connections between JS and Python that let us see and change the value of this slider using its `value` parameter:

In [ ]:
mag_slider.value

In [ ]:
mag_slider.value = 7

#### Exercise

Try moving the slider around and rerunning the `mag_slider.value` above to access the current slider value. As you can see, you can easily get the value of any widget to use in subsequent cells, but you'd need to re-run any cell that accesses that value for it to get updated. Getting output to update automatically is where `.rx` comes in.

## Panel `.rx`

`.rx` expressions provide an easy way to connect widgets directly into an expression you want to control.

First, let's read in our data:

In [ ]:
%%time
df = pd.read_parquet(pathlib.Path('../data/earthquakes-projected.parq'))
df.index = df.index.tz_localize(None)

Now, let's do a little filtering that we might want to control with such a widget, such as selecting the highest-magnitude events:

In [ ]:
WEB_MERCATOR_LIMITS = (-20037508.342789244, 20037508.342789244)

df2 = df[['mag', 'depth', 'latitude', 'longitude', 'place', 'type']][df['northing'] < WEB_MERCATOR_LIMITS[1]]

df2[df2['mag'] > 5].head()

What if instead of '5', we want the output above always to reflect the current value of `mag_slider`? It's trivially easy to do that with `.rx`, passing in a widget almost anywhere we want in a pipeline:

In [ ]:
rdf = pn.rx(df2)

rdf[rdf['mag'] > mag_slider].head()

(Compare the previous cell to the one before it, to see that `5` has been replaced with `mag_slider`.) `pn.rx` wraps around nearly any Python object (such as the Pandas DataFrame used here) while letting you provide Panel widgets almost anywhere you'd otherwise be using a number, string, list, tuple, or other Python literal or variable. Basically, if Panel has a [widget](https://panel.holoviz.org/reference/index.html#widgets) for it, you can use it! 

The way it works is that the `rx` object no longer immediately calls the methods and operations you apply to it; instead it stores a copy of your pipeline and dynamically replays the pipeline whenever that widget changes. `rx` supports just about any output you might want to get out of such a pipeline, such as text or numbers:

In [ ]:
rdf[rdf['mag'] > mag_slider].shape

Or Matplotlib plots:

In [ ]:
rdf[rdf['mag'] > mag_slider].plot(y='depth', kind='hist', bins=np.linspace(0, 50, 51), figsize=(6, 3))

Each time you drag the widget and release it (thanks to `throttled=True` above), hvPlot replays the pipeline and updates the output shown.

Of course, `.rx` also supports `.hvplot()`, here with a new copy of a widget so that it will be independent of the other cells above:

In [ ]:
mag_slider2 = pn.widgets.FloatSlider(name='Minimum magnitude', start=0, end=9, value=6)

rdf[rdf['mag'] > mag_slider2].hvplot(y='depth', kind='hist', bins=np.linspace(0, 50, 51))

You can see that the depth distribution varies dramatically as you vary the minimum magnitude, with the lowest magnitude events apparently only detectable at short depths. There also seems to be some artifact at depth 10, which is the largest bin regardless of the filtering for all but the largest magnitudes.

## Date widgets

A `.rx()` pipeline can contain any number of widgets, including any from the Panel [reference gallery](https://panel.holoviz.org/reference/index.html#widgets). For instance, let's make a widget to specify a date range covering the dates found in this data:

In [ ]:
date = pn.widgets.DatetimeRangeSlider(name='Date', start=df.index[0], end=df.index[-1])
date

Now we can access the value of this slider:

In [ ]:
date.value

As this widget is specifying a range, this time the value is returned as a tuple. If you prefer, you can get the components of the tuple directly via the `value_start` and `value_end` parameters respectively:

In [ ]:
f'Start is at {date.value_start} and the end is at {date.value_end}'

Once again, try specifying different ranges with the widgets and rerunning the cell above.

Now let's use this widget to expand our expression to filter by date as well as magnitude:

In [ ]:
mag = pn.widgets.FloatSlider(name='Minimum magnitude', start=0, end=9, value=6)

filtered = rdf[
    (rdf['mag'] > mag) &
    (rdf.index >= date.param.value_start) &
    (rdf.index <= date.param.value_end)]

filtered.head()

You can now use either the magnitude or the date range (or both) to filter the data, and the output will update. Note that here you want to move the start date of the range slider rather than the end; otherwise, you may not see the table change because the earthquakes are displayed in date order.

#### Exercise

For readability, seven columns were chosen before displaying the `DataFrame`. Have a look at `df.columns` and pick a different set of columns for display, and try filtering on one of them.

## Functions as inputs

Quite often the data structure you want to explore in a pipeline, may itself be the outcome of another pipeline. It may for instance be a Pandas Dataframe created by extracting and transforming the output of a database or an API call, or it could be the dynamic output of some simulation or pre-processing. You can wrap function calls with `pn.rx`, but Panel provides a more explicit way to bind reactive arguments to functions, called `pn.bind`. With `bind` you can start with an arbitrary custom function that returns the data structure you want to explore, and then bind widgets to that function's arguments. Then when one of those widgets changes, the function will get called to get the updated output.

To keep this example self-contained, we'll illustrate this process using a simple function that filters the earthquakes dataset by event type and returns a DataFrame. Of course, this function could include _any_ computation that returns a DataFrame, including selecting data files on disk or making a query to a database.

In [ ]:
def input_function(event_type):
    df2 = df[['mag', 'depth', 'latitude', 'longitude', 'place', 'type']]
    return df2[df2['type'] == event_type]

We can then create a Panel `Select` widget with a few options and bind it to the `input_function`. Calling `.rx()` on the bound object is what allows it to be used in a reactive pipeline, as we previously did with `rdf`.

In [ ]:
event_types = pn.widgets.Select(options=['earthquake', 'quarry blast', 'explosion', 'ice quake'])

inputrx = pn.bind(input_function, event_types).rx()

In [ ]:
inputrx[inputrx['mag'] > mag].head(2)

## .rx() and HoloViews 

`.rx()` lets you work naturally with the compositional HoloViews plots provided by `.hvplot()`. Here, let's combine such plots using the HoloViews `+` operator:

In [ ]:
mag_hist   = filtered.hvplot(y='mag',   kind='hist', width=300)
depth_hist = filtered.hvplot(y='depth', kind='hist', width=300)

mag_hist + depth_hist

These are the same two histograms we saw earlier, but now we can filter them on data dimensions like `time` that aren't even explicitly shown in the plot, using the Panel widgets.

## Filtering earthquakes on a map

To display the earthquakes on a map, we will first create a subset of the data to make it quick to update without needing Datashader.:

In [ ]:
subset_df = df[
            (df.northing <  WEB_MERCATOR_LIMITS[1]) &
            (df.mag      >  4) &
            (df.index    >= '2017-01-01') &
            (df.index    <= '2018-01-01')]

Now we can make a new reactive `DataFrame` from this new subselection:

In [ ]:
subset = pn.rx(subset_df)

And now we can declare our widgets and use them to filter the reactive `DataFrame` as before:

In [ ]:
date_subrange = pn.widgets.DatetimeRangeSlider(
    name='Date', start=subset_df.index[0], end=subset_df.index[-1])

mag_subrange = pn.widgets.FloatSlider(name='Magnitude', start=3, end=9, value=3)

filtered_subrange = subset[
    (subset.mag   >  mag_subrange) &
    (subset.index >= date_subrange.param.value_start) &
    (subset.index <= date_subrange.param.value_end)]

Now we can plot the earthquakes on an ESRI tilesource, including the filtering widgets as follows:

In [ ]:
geo = filtered_subrange.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500, tiles='ESRI')

geo

## Terminating reactive expressions

The examples above all illustrate cases where you can display the output of `.rx()` and not worry about its type, since _eventually_ you'll get an object of the type you expect (a number, Matplotlib plot, HoloViews object, etc.), when it comes time to display something. But sometimes the actual return type from your expression does matter, and until it's displayed, a reactive expression is an `rx` object rather than a DataFrame or a HoloViews object or whatever your expression evaluates to:

In [ ]:
type(geo)

Panel will always render the `rx` object dynamically, but because a reactive expression could return literally anything in Python, such rendering is not always efficient -- what if only a small part of the result is changing, and you want only that bit updated, not redrawing the entire figure?

Or what if you need to work with some part of the reactive pipeline for your own purposes, e.g. to feed it to some function or object that does not understand `rx` objects?

For both of these reasons you may want resolve the expression into an underlying object, which we call "termination" of the expression.

For instance, let's create magnitude and depth histograms on this subset of the data as in an earlier notebook and see if we can enable linked selections on them:

In [ ]:
mag_subhist   = filtered_subrange.hvplot(y='mag',   kind='hist', responsive=True, min_height=200)
depth_subhist = filtered_subrange.hvplot(y='depth', kind='hist', responsive=True, min_height=200)

combined = mag_subhist + depth_subhist

pn.Row(combined, height=400)

Note that this looks like a HoloViews layout with some widgets, because the expression evaluates to such a layout when displayed, but this object is *not* a HoloViews object. Instead it is still an `rx` object:

In [ ]:
type(combined)

`link_selections` does not currently understand `rx` objects, and so it will raise an exception when given one. Luckily, a HoloViews `DynamicMap` _does_ understand `rx`, so if we need a HoloViews `Layout` for calling `link_selections`, we can build a layout from the constituent objects by passing each expression to a HoloViews `DynamicMap`:

In [ ]:
layout = hv.DynamicMap(mag_subhist) + hv.DynamicMap(depth_subhist)
layout

This is now a HoloViews object, so we can use it with `link_selections`:

In [ ]:
print(type(layout))

ls = hv.link_selections.instance()
ls(hv.DynamicMap(mag_subhist) + hv.DynamicMap(depth_subhist))

You can use the box selection tool to see how selections compare between these plots. However, you will note that the separate widgets are not automatically displayed for terminated expressions like this. To address this, we can obtain the widgets from the panel `ReactiveExpr` pane:

In [ ]:
pn.param.ReactiveExpr(filtered_subrange).widgets

Lastly we can also pass expressions directly to Panel components, assuming the value returned by the expression matches the type expected by Panel. As an example we can compute the maximum magnitude of our filtered data and pass it to a `Number` indicator:

In [ ]:
pn.indicators.Number(value=filtered_subrange.mag.max(), name='Magnitude')

Similarly we could pass an expression returning a `DataFrame` to the `Tabulator` widget or an expression returning a matplotlib plot to a `Matplotlib` pane. Having a particular component resolve our expressions allows Panel to efficiently re-render the component.

## Conclusion

Using the techniques above, you can build up a collection of plots and other outputs with Panel widgets to control individual bits of computation and display. 

What if you want to collect these pieces and put them together into a standalone app or dashboard? If so, then the next tutorial will show you how to do so!